In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=316f609205fd17c5a09606442884b51b8196a96558f4e68b827543f18ad4acdf
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Window


In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
data = [(1,'Junior',10000),
        (2,'Junior',15000),
        (3,'Junior',40000),
        (4,'Senior',16000),
        (5,'Senior',20000),
        (6,'Senior',50000)
        ]


In [ ]:
df = spark.createDataFrame(data, ['emp_id','experience','salary'])

In [ ]:
df.printSchema()

root
 |-- emp_id: long (nullable = true)
 |-- experience: string (nullable = true)
 |-- salary: long (nullable = true)



In [ ]:
window_senior = Window.partitionBy('experience').orderBy('salary')
window_junior = Window.partitionBy('experience').orderBy('salary')

In [ ]:
df = df.withColumn('senior_cumulative', F.sum('salary').over(window_senior).cast(T.IntegerType()))\
       .withColumn('junior_cumulative', F.sum('salary').over(window_junior).cast(T.IntegerType()))

In [ ]:
seniors_df = df.filter((F.col('experience') == 'Senior') & (F.col('senior_cumulative') <= 70000)).select('emp_id','salary')

In [ ]:
junior_budget = seniors_df.agg(F.sum('salary')).first()[0]


In [ ]:
juniors_df = df.filter((F.col('experience') == 'Junior') & (F.col('junior_cumulative') <= junior_budget)).select('emp_id','salary')

In [ ]:
total_df = seniors_df.union(juniors_df).show()

+------+------+
|emp_id|salary|
+------+------+
|     4| 16000|
|     5| 20000|
|     1| 10000|
|     2| 15000|
+------+------+

